In [1]:
! pip install transformers  datasets  peft torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import numpy as np


In [8]:
device ="cuda"
torch.device(device)

device(type='cuda')

In [3]:
# load dataset
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")
train_dataset = dataset["train"]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [4]:
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
# preprocess dataset

def preprocess_function(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=128)
    return tokenized_inputs

tokenized_dataset = train_dataset.map(preprocess_function, batched=True ,remove_columns=['text'])


Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

In [6]:
tokenized_dataset.set_format("torch")

In [7]:
# configure lora

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["c_attn", "c_proj"],   # GPT-2 attention and projection layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [9]:
model = get_peft_model(model ,lora_config)
model.print_trainable_parameters()

trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [10]:
# training arguments

training_args = TrainingArguments(
    output_dir="./gpt2_results",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    logging_steps=100,
    learning_rate= 2e-4,
    fp16=torch.cuda.is_available() )

In [11]:
# initialize trainer
trainer = Trainer(
    model = model ,
    args = training_args,
    train_dataset = tokenized_dataset,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [12]:
trainer.train

<bound method Trainer.train of <transformers.trainer.Trainer object at 0x79210bae9ed0>>

In [13]:
# save the  model

model.save_pretrained("./gpt2_results")
tokenizer.save_pretrained("./gpt2_results")

('./gpt2_results/tokenizer_config.json',
 './gpt2_results/special_tokens_map.json',
 './gpt2_results/vocab.json',
 './gpt2_results/merges.txt',
 './gpt2_results/added_tokens.json')

In [14]:
# evaluation

model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPT2LMHeadModel(
      (transformer): GPT2Model(
        (wte): Embedding(50257, 768)
        (wpe): Embedding(1024, 768)
        (drop): Dropout(p=0.1, inplace=False)
        (h): ModuleList(
          (0-11): 12 x GPT2Block(
            (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (attn): GPT2Attention(
              (c_attn): lora.Linear(
                (base_layer): Conv1D(nf=2304, nx=768)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2304, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
          

In [17]:
# generate example
prompt = "The future of artificial intelligence is"

inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(**inputs ,max_length = 50  , num_return_sequences = 1 , do_sample = True )

generated_text = tokenizer.decode(outputs[0] ,skip_special_tokens=True)

print(f"generated text : {generated_text}")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


generated text : The future of artificial intelligence is unknown."

"They have a lot to learn," said CIO Richard Mather, "but the endgame for AI will always be to become a master scientist and then turn around again. If we achieve that


In [19]:
! pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.0 MB/s eta 0:00:00


In [22]:
import ipywidgets as widgets
from IPython.display import display, HTML
# Define the text generation function
def generate_text(prompt, max_length, temperature, top_k, top_p):
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    # Generate text with sampling enabled
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=1,
        do_sample=True,  # Enable sampling
        top_k=top_k,
        top_p=top_p,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# Create an interactive UI with ipywidgets
# Input prompt
prompt_input = widgets.Text(
    value="The future of artificial intelligence is",
    description="Prompt:",
    layout={'width': '500px'}
)

# Slider for max length
max_length_slider = widgets.IntSlider(
    value=50,
    min=20,
    max=200,
    step=1,
    description="Max Length:",
    layout={'width': '500px'}
)

# Slider for temperature
temperature_slider = widgets.FloatSlider(
    value=0.7,
    min=0.1,
    max=2.0,
    step=0.1,
    description="Temperature:",
    layout={'width': '500px'}
)

# Slider for top_k
top_k_slider = widgets.IntSlider(
    value=50,
    min=10,
    max=100,
    step=1,
    description="Top K:",
    layout={'width': '500px'}
)

# Slider for top_p
top_p_slider = widgets.FloatSlider(
    value=0.95,
    min=0.5,
    max=1.0,
    step=0.01,
    description="Top P:",
    layout={'width': '500px'}
)

# Button to generate text
generate_button = widgets.Button(
    description="Generate Text",
    button_style="success",
    tooltip="Click to generate text"
)

# Output area for generated text
output = widgets.Output()

# Define button click event
def on_generate_button_clicked(b):
    with output:
        output.clear_output()
        prompt = prompt_input.value
        max_length = max_length_slider.value
        temperature = temperature_slider.value
        top_k = top_k_slider.value
        top_p = top_p_slider.value
        if prompt.strip() == "":
            print("Error: Please enter a prompt.")
        else:
            print("Generating text...")
            generated_text = generate_text(prompt, max_length, temperature, top_k, top_p)
            print("Generated Text:")
            print(generated_text)

generate_button.on_click(on_generate_button_clicked)

# Display the UI
display(HTML("<h2>Text Generation with Fine-Tuned GPT-2</h2>"))
display(prompt_input)
display(max_length_slider)
display(temperature_slider)
display(top_k_slider)
display(top_p_slider)
display(generate_button)
display(output)

# Save the model to Google Drive (optional)
# Uncomment the following lines to save the model to Google Drive
"""
from google.colab import drive
drive.mount('/content/drive')
!cp -r ./gpt2_results /content/drive/MyDrive/gpt2_results_backup
print("Model saved to Google Drive!")
"""

Text(value='The future of artificial intelligence is', description='Prompt:', layout=Layout(width='500px'))

IntSlider(value=50, description='Max Length:', layout=Layout(width='500px'), max=200, min=20)

FloatSlider(value=0.7, description='Temperature:', layout=Layout(width='500px'), max=2.0, min=0.1)

IntSlider(value=50, description='Top K:', layout=Layout(width='500px'), min=10)

FloatSlider(value=0.95, description='Top P:', layout=Layout(width='500px'), max=1.0, min=0.5, step=0.01)

Button(button_style='success', description='Generate Text', style=ButtonStyle(), tooltip='Click to generate te…

Output()

'\nfrom google.colab import drive\ndrive.mount(\'/content/drive\')\n!cp -r ./gpt2_results /content/drive/MyDrive/gpt2_results_backup\nprint("Model saved to Google Drive!")\n'

In [23]:
!pip install  streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.5 MB/s eta 0:00:00


In [28]:
from google.colab import userdata
authkey =userdata.get('ngrokauth')

In [36]:

# Interactive loop for text generation
def interactive_text_generation():
    print("=== Text Generation with Fine-Tuned GPT-2 ===")
    print("Enter a prompt to generate text. Type 'exit' to stop.")
    print("You can also specify generation parameters (or press Enter to use defaults).")

    while True:
        # Get the prompt
        prompt = input("Prompt (e.g., 'The future of artificial intelligence is'): ")
        if prompt.lower() == 'exit':
            print("Exiting text generation.")
            break
        if prompt.strip() == "":
            print("Error: Please enter a prompt.")
            continue

        # Get generation parameters (with defaults)
        max_length_input = input("Max Length (default 50, range 20-200): ") or "50"
        temperature_input = input("Temperature (default 0.7, range 0.1-2.0): ") or "0.7"
        top_k_input = input("Top K (default 50, range 10-100): ") or "50"
        top_p_input = input("Top P (default 0.95, range 0.5-1.0): ") or "0.95"

        # Validate and convert inputs
        try:
            max_length = int(max_length_input)
            if not (20 <= max_length <= 200):
                print("Max Length must be between 20 and 200. Using default (50).")
                max_length = 50

            temperature = float(temperature_input)
            if not (0.1 <= temperature <= 2.0):
                print("Temperature must be between 0.1 and 2.0. Using default (0.7).")
                temperature = 0.7

            top_k = int(top_k_input)
            if not (10 <= top_k <= 100):
                print("Top K must be between 10 and 100. Using default (50).")
                top_k = 50

            top_p = float(top_p_input)
            if not (0.5 <= top_p <= 1.0):
                print("Top P must be between 0.5 and 1.0. Using default (0.95).")
                top_p = 0.95
        except ValueError:
            print("Invalid input. Using default values.")
            max_length, temperature, top_k, top_p = 50, 0.7, 50, 0.95

        # Generate and display text
        print("\nGenerating text...")
        generated_text = generate_text(prompt, max_length, temperature, top_k, top_p)
        print("Generated Text:")
        print(generated_text)
        print("\n" + "="*50 + "\n")

# Run the interactive text generation
interactive_text_generation()

=== Text Generation with Fine-Tuned GPT-2 ===
Enter a prompt to generate text. Type 'exit' to stop.
You can also specify generation parameters (or press Enter to use defaults).
Prompt (e.g., 'The future of artificial intelligence is'): hi 
Max Length (default 50, range 20-200): 50
Temperature (default 0.7, range 0.1-2.0): 0.1
Top K (default 50, range 10-100): 10
Top P (default 0.95, range 0.5-1.0): 0.95

Generating text...
Generated Text:
hi ಠ_ಠ ಠ_ಠ ಠ_ಠ ಠ_ಠ ಠ_ಠ ಠ_ಠ 


Prompt (e.g., 'The future of artificial intelligence is'): how are you 
Max Length (default 50, range 20-200): 50 
Temperature (default 0.7, range 0.1-2.0): 0.7
Top K (default 50, range 10-100): 50
Top P (default 0.95, range 0.5-1.0): 0.95

Generating text...
Generated Text:
how are you ?"

"There's no such thing as a 'just like' or 'just like' in this world."

"So, you're saying that it's okay to be like this, right?"

"




KeyboardInterrupt: Interrupted by user